In [1]:
%load_ext iminizinc

<IPython.core.display.Javascript object>

MiniZinc to FlatZinc converter, version 2.5.5
Copyright (C) 2014-2021 Monash University, NICTA, Data61


# Scheduling

In this notebook, we explore how to solve a resource constrained project scheduling problem (RCPSP).

The problem is made of $M$ activities that have precedence constraints. That means that if activity $j\in[1,M]$ is a successor of activity $i\in[1,M]$, then activity $i$ must be completed before activity $j$ can be started

On top of these constraints, each project is assigned a set of K renewable resources where each resource $k$ is available in $R_{k}$ units for the entire duration of the project. Each activity may require one or more of these resources to be completed. While scheduling the activities, the daily resource usage for resource $k$ can not exceed $R_{k}$ units.

Each activity $j$ takes $d_{j}$ time units to complete.

The overall goal of the problem is usually to minimize the makespan,

(Variants: mix of renewable and non-renewable resources / multi-mode)


## Problem definition

Let start with a simple problem which has 5 tasks to execute:

 - Task 0 has a duration of 1 hour
 - Task 1 depends on 0 and has a duration of 3 hours
 - Task 2 depends on 0 and has a duration of 1 hours
 - Task 3 depends on 2 and has a duration of 3 hours
 - Task 4 depends on 1 & 3 and has a duration of 2 hours

This can be reworked saying that:
 - Task 0 has 2 successors: task 1 and task 2
 - Task 1 has one successor: task 4
 - Task 2 has one successor: task 3
 - Task 3 has one successor: task 4
 - Task 4 has no successor

## Domain construction

In [1]:
from typing import Any, Dict, List, Optional, Set, Union

from skdecide.builders.domain.scheduling.scheduling_domains import SingleModeRCPSP, SchedulingObjectiveEnum
from skdecide import Distribution, DiscreteDistribution

from skdecide.builders.domain.scheduling.modes import ModeConsumption, ConstantModeConsumption
from skdecide import rollout_episode

class SimpleRCPSPDomain(SingleModeRCPSP):

    def _get_objectives(self) -> List[SchedulingObjectiveEnum]:
        return [SchedulingObjectiveEnum.MAKESPAN]

    def __init__(self):
        self.initialize_domain()

    def _get_max_horizon(self) -> int:
        return 50

    def _get_successors(self) -> Dict[int, List[int]]:
        return {0: [1, 2], 1: [4], 2: [3], 3: [4], 4: []}

    def _get_tasks_ids(self) -> Union[Set[int], Dict[int, Any], List[int]]:
        return {0, 1, 2, 3, 4}

    def _get_tasks_mode(self) -> Dict[int, ModeConsumption]:
        return {
            0: ConstantModeConsumption({'r1': 1,}),
            1: ConstantModeConsumption({'r1': 1,}),
            2: ConstantModeConsumption({'r1': 1,}),
            3: ConstantModeConsumption({'r1': 1,}),
            4: ConstantModeConsumption({'r1': 1,})
        }

    def _get_resource_types_names(self) -> List[str]:
        return ['r1']

    def _get_task_duration(self, task: int, mode: Optional[int] = 1, progress_from: Optional[float] = 0.) -> int:
        all_durations = {0: 1, 1: 3, 2: 1, 3: 3, 4: 2}
        return all_durations[task]

    def _get_original_quantity_resource(self, resource: str, **kwargs) -> int:
        all_resource_quantities = {'r1': 10_000}
        return all_resource_quantities[resource]


In [2]:
domain = SimpleRCPSPDomain()
domain.set_inplace_environment(False)
state = domain.get_initial_state()

## Let's analyze the graph 

In [3]:
graph = domain.compute_graph()

### Show the graph

In [4]:
def to_json(graph):
    pass

# plot graph in javascript

## Let's use the LazyAstar to solve this problem

In [5]:
from skdecide.hub.solver.lazy_astar import LazyAstar

solver = LazyAstar(from_state=state, heuristic=None)
solver.solve(domain_factory=lambda: domain)

# Load a sm file

In [2]:
from skdecide.hub.domain.rcpsp.rcpsp_sk_parser import load_domain
from skdecide.hub.domain.rcpsp.rcpsp_sk import RCPSP
from skdecide.discrete_optimization.rcpsp.solver.rcpsp_pile import PileSolverRCPSP, PileSolverRCPSP_Calendar, GreedyChoice
from skdecide.discrete_optimization.rcpsp.solver.rcpsp_lp_solver import LP_RCPSP

In [3]:
big_domain: RCPSP = load_domain("j301_1.sm")
big_domain.set_inplace_environment(False)

In [8]:
do_work = False

if (do_work):
    state = big_domain.get_initial_state()

    # This is never ending
    # solver = LazyAstar(from_state=state, heuristic=None)
    # solver.solve(domain_factory=lambda: big_domain)

    solver = PileSolverRCPSP(rcpsp_model=big_domain)
    results = solver.solve(greedy_choice=GreedyChoice.MOST_SUCCESSORS)

    best_solution = results.get_best_solution()
    fitnesses = big_domain.evaluate(best_solution)

In [7]:
from skdecide.hub.solver.do_solver.do_solver_scheduling import DOSolver, PolicyMethodParams, SolvingMethod, BasePolicyMethod

In [10]:
%%minizinc -a

state = big_domain.get_initial_state()

# base_policy_method=BasePolicyMethod.FOLLOW_GANTT
base_policy_method=BasePolicyMethod.SGS_PRECEDENCE

policy_method_params = PolicyMethodParams(base_policy_method=base_policy_method, delta_index_freedom=0, delta_time_freedom=0)

#method = SolvingMethod.CP              # BOOM
#method = SolvingMethod.LP
#method = SolvingMethod.GA
#method = SolvingMethod.PILE            # BOOM
#method = SolvingMethod.LS
#method = SolvingMethod.LNS_LP          # BOOM
#method = SolvingMethod.LNS_CP          # BOOM
#method = SolvingMethod.LNS_CP_CALENDAR # BOOM 

# We want to focus on CP and LNS_CP
method = SolvingMethod.CP

solver = DOSolver(policy_method_params=policy_method_params, method=method, dict_params={"verbose": False})

solver.solve(domain_factory=lambda: big_domain)

/var/folders/ds/qwbvxlpn2hd_69wn9ypzl_c80000gn/T/tmpqm24ynl6/model.mzn:2.19:
state = big_domain.get_initial_state()
                  ^
Error: syntax error, unexpected invalid token, expecting end of file


| Method               | Time (s) | Span   |
| ---                  | ---      | ---    |
| SolvingMethod.LP     | 19       | 43     |
| SolvingMethod.CP     | -        |        |
| SolvingMethod.GA     | 13       | 49     |
| SolvingMethod.LS     | 3        | 43     |
| SolvingMethod.PILE   | -        |        |
| SolvingMethod.LNS_LP | -        |        | 

# Solvers

In [6]:
!cat /var/folders/ds/qwbvxlpn2hd_69wn9ypzl_c80000gn/T/tmpljnn5rl5/model.mzn

cat: /var/folders/ds/qwbvxlpn2hd_69wn9ypzl_c80000gn/T/tmpljnn5rl5/model.mzn: No such file or directory


In [10]:
from skdecide.discrete_optimization.rcpsp.rcpsp_solvers import *

In [11]:
print(solvers.keys())

dict_keys(['lp', 'greedy', 'cp', 'lns', 'lns-lp', 'lns-cp', 'ls', 'ga', 'lns-cp-calendar'])


In [12]:
# LP_RCPSP: Linear Programming RCPSP
# Pile_
# CP_
# LNS_
# LS_ : Local Search
# GA_ : Genetic Algorithm 

for solver in solvers:
    for option in solvers[solver]:
        klass = option[0]
        option_dict = option[1]
        print(f"{klass} {klass.__name__}: {' '.join(option_dict.keys())}")

<class 'skdecide.discrete_optimization.rcpsp.solver.rcpsp_lp_solver.LP_RCPSP'> LP_RCPSP: lp_solver parameters_milp
<class 'skdecide.discrete_optimization.rcpsp.solver.rcpsp_lp_solver.LP_MRCPSP'> LP_MRCPSP: lp_solver parameters_milp
<class 'skdecide.discrete_optimization.rcpsp.solver.rcpsp_pile.PileSolverRCPSP'> PileSolverRCPSP: greedy_choice
<class 'skdecide.discrete_optimization.rcpsp.solver.rcpsp_pile.PileSolverRCPSP_Calendar'> PileSolverRCPSP_Calendar: greedy_choice
<class 'skdecide.discrete_optimization.rcpsp.solver.cp_solvers.CP_RCPSP_MZN'> CP_RCPSP_MZN: cp_solver_name parameters_cp
<class 'skdecide.discrete_optimization.rcpsp.solver.cp_solvers.CP_MRCPSP_MZN'> CP_MRCPSP_MZN: cp_solver_name parameters_cp
<class 'skdecide.discrete_optimization.rcpsp.solver.rcpsp_lp_lns_solver.LNS_LP_RCPSP_SOLVER'> LNS_LP_RCPSP_SOLVER: nb_iteration_lns lp_solver
<class 'skdecide.discrete_optimization.rcpsp.solver.rcpsp_cp_lns_solver.LNS_CP_RCPSP_SOLVER'> LNS_CP_RCPSP_SOLVER: nb_iteration_lns
<class '